In [40]:
import os
import sys

sys.path.append("..")
import fiftyone as fo

from fiftyone.utils.huggingface import load_from_hub
from utils.data_loader import (
    FiftyOneTorchDatasetCOCO,
    FiftyOneTorchDatasetCOCOFilepaths,
    TorchToHFDatasetCOCO,
)

from torch.utils.data import DataLoader

In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
tokens = {}
with open("/home/dbogdoll/mcity_data_engine/.secret", "r") as file:
    for line in file:
        key, value = line.strip().split("=")
        tokens[key] = value

os.environ["HF_TOKEN"] = tokens["HF_TOKEN"]

try:
    dataset_v51 = load_from_hub("dbogdollumich/mcity_fisheye_v51")
except:
    dataset_v51 = fo.load_dataset("dbogdollumich/mcity_fisheye_v51")

print(dataset_v51.stats)
print(dataset_v51.summary)
print(dataset_v51.name)
print(len(dataset_v51))

Loading dataset
<bound method Dataset.stats of Name:        dbogdollumich/mcity_fisheye_v51
Media type:  image
Num samples: 2744
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    location:         fiftyone.core.fields.StringField
    name:             fiftyone.core.fields.StringField
    timestamp:        fiftyone.core.fields.DateTimeField>
<bound method Dataset.summary of Name:        dbogdollumich/mcity_fisheye_v51
Media type:  image
Num samples: 2744
Persistent:  False
Tags:

In [43]:
sample = dataset_v51.first()
print(sample)
sample2 = dataset_v51[sample.filepath]

ground_truth = sample["ground_truth"]
print(ground_truth)
print(len(ground_truth["detections"]))

for detection in ground_truth["detections"]:
    print(detection)
    print(detection.bounding_box)

<Sample: {
    'id': '670d7c398fccaa34bc6b2bbe',
    'media_type': 'image',
    'filepath': '/home/dbogdoll/fiftyone/huggingface/hub/dbogdollumich/mcity_fisheye_v51/data/Huron_Plymouth-Geddes_Huron_gs_Geddes_Huron1__2023-03-28__11__0_gs_Geddes_Huron1__2023-03-28__11__0__2023-03-28_11-06-01-107336.jpg',
    'tags': ['train'],
    'metadata': <ImageMetadata: {
        'size_bytes': 263379,
        'mime_type': 'image/jpeg',
        'width': 1280,
        'height': 960,
        'num_channels': 3,
    }>,
    'created_at': datetime.datetime(2024, 10, 31, 21, 55, 4, 181000),
    'last_modified_at': datetime.datetime(2024, 10, 31, 21, 55, 4, 181000),
    'ground_truth': <Detections: {
        'detections': [
            <Detection: {
                'id': '670d7c398fccaa34bc6b2bac',
                'attributes': {},
                'tags': [],
                'label': 'car',
                'bounding_box': [0.24241120000000002, 0.52306885, 0.0265376, 0.0737743],
                'mask': None,

In [44]:
batch_size = 12
# dataset_v51 = dataset_v51.take(500)

In [45]:
torch_dataset_filepaths = FiftyOneTorchDatasetCOCOFilepaths(dataset_v51)
torch_dataloader_filepaths = DataLoader(
    torch_dataset_filepaths,
    batch_size=batch_size,
    num_workers=32,
    pin_memory=True,
    shuffle=False,
    prefetch_factor=2,
    persistent_workers=True,
)

Processing Voxel51 dataset: 100%|██████████| 2744/2744 [00:02<00:00, 1089.91it/s]


In [46]:
torch_dataset_managed = FiftyOneTorchDatasetCOCO(dataset_v51)
torch_dataloader_managed = DataLoader(
    torch_dataset_managed,
    batch_size=batch_size,
    num_workers=32,
    pin_memory=True,
    shuffle=False,
    prefetch_factor=2,
    collate_fn=lambda batch: list(zip(*batch)),
)

print(torch_dataset_managed.get_classes())
print(torch_dataset_managed.get_splits())

Processing Voxel51 dataset: 100%|██████████| 2744/2744 [00:05<00:00, 515.60it/s]

['car', 'truck', 'bus', 'trailer', 'motorbike/cycler', 'pedestrian', 'van', 'pickup']
{'val', 'train'}


In [47]:
converter = TorchToHFDatasetCOCO(torch_dataset_managed)
hf_dataset = converter.convert()

Generating validation split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [48]:
import fiftyone.utils.coco as fouc
import torch


def target_batch_to_detections(targets, dataset, gt_field="ground_truth"):
    classes = dataset_v51.default_classes
    batch_targets = []
    samples = [dataset[filepath] for filepath in targets]
    for sample in samples:
        metadata = sample.metadata
        id = sample.id
        boxes = []
        labels = []
        area = []
        iscrowd = []
        for detection in sample[gt_field]["detections"]:
            category_id = classes.index(detection.label)
            coco_obj = fouc.COCOObject.from_label(
                detection,
                metadata,
                category_id=category_id,
            )
            x, y, w, h = coco_obj.bbox
            boxes.append([x, y, w, h])
            labels.append(coco_obj.category_id)
            area.append(coco_obj.area)
            iscrowd.append(coco_obj.iscrowd)

        target_dict = {}
        target_dict["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target_dict["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target_dict["image_id"] = id
        target_dict["area"] = torch.as_tensor(area, dtype=torch.float32)
        target_dict["iscrowd"] = torch.as_tensor(iscrowd, dtype=torch.int64)
        batch_targets.append(target_dict)
    return batch_targets

In [49]:
from transformers import (
    AutoProcessor,
    AutoConfig,
    AutoTokenizer,
    AutoModelForZeroShotObjectDetection,
)

model_name = "google/owlv2-base-patch16-ensemble"
texts = [
    "car",
    "truck",
    "bus",
    "trailer",
    "motorbike/cycler",
    "pedestrian",
    "van",
    "pickup",
]

config = AutoConfig.from_pretrained(model_name)
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_name).to(device)

os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [50]:
batch_classes = texts * batch_size
tokenized_text = processor.tokenizer(
    batch_classes, padding="max_length", return_tensors="pt"
).to(device)

In [ ]:
n_samples = len(dataset_v51)
iterations = n_samples / batch_size
torch.cuda.empty_cache()
steps = 0
for batch_idx, (images, targets) in enumerate(torch_dataloader_managed):
    print(batch_idx)
    steps += 1
print("Steps: ", steps)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
Steps:  229


In [ ]:
torch.cuda.empty_cache()

for batch_idx, (images, targets) in enumerate(torch_dataloader_filepaths):
    targets = target_batch_to_detections(targets, dataset_v51)
    target_sizes = [tuple(img.shape[1:]) for img in images]
    inputs = processor(text=None, images=images, return_tensors="pt").to(device)
    inputs.update(tokenized_text)
    with torch.amp.autocast("cuda"):
        with torch.no_grad():
            outputs = model(**inputs)
    results = processor.post_process_object_detection(
        outputs=outputs,
        threshold=0.2,
        target_sizes=target_sizes,
    )
    for result, target in zip(results, targets):
        boxes, scores, labels = result["boxes"], result["scores"], result["labels"]

        detections = []
        for box, score, label in zip(boxes, scores, labels):

            # Get image size (ID is stored in annotation)
            sample = dataset_v51[target["image_id"]]
            img_width = sample.metadata.width
            img_height = sample.metadata.height

            # Convert bbox to V51 type
            label = texts[label]
            top_left_x = box[0].item() / img_width
            top_left_y = box[1].item() / img_height
            box_width = (box[2].item() - box[0].item()) / img_width
            box_height = (box[3].item() - box[1].item()) / img_height

            detection = fo.Detection(
                label=label,
                bounding_box=[
                    top_left_x,
                    top_left_y,
                    box_width,
                    box_height,
                ],
                confidence=score.item(),
            )

            detections.append(detection)

        # Attach label to V51 dataset
        sample = dataset_v51[target["image_id"]]
        sample["prediction"] = fo.Detections(detections=detections)
        sample.save()

In [ ]:
torch.cuda.empty_cache()

for batch_idx, (images, targets) in enumerate(torch_dataloader_managed):
    target_sizes = [tuple(img.shape[1:]) for img in images]
    inputs = processor(text=None, images=images, return_tensors="pt").to(device)
    inputs.update(tokenized_text)
    with torch.amp.autocast("cuda"):
        with torch.no_grad():
            outputs = model(**inputs)
    results = processor.post_process_object_detection(
        outputs=outputs,
        threshold=0.2,
        target_sizes=target_sizes,
    )
    for result, target in zip(results, targets):
        boxes, scores, labels = result["boxes"], result["scores"], result["labels"]

        detections = []
        for box, score, label, img_size in zip(boxes, scores, labels, target_sizes):

            img_width = img_size[1]
            img_height = img_size[0]

            # Convert bbox to V51 type
            label = texts[label]
            top_left_x = box[0].item() / img_width
            top_left_y = box[1].item() / img_height
            box_width = (box[2].item() - box[0].item()) / img_width
            box_height = (box[3].item() - box[1].item()) / img_height

            detection = fo.Detection(
                label=label,
                bounding_box=[
                    top_left_x,
                    top_left_y,
                    box_width,
                    box_height,
                ],
                confidence=score.item(),
            )

            detections.append(detection)

        # Attach label to V51 dataset
        sample = dataset_v51[target["image_id"]]
        sample["prediction_managed"] = fo.Detections(detections=detections)
        sample.save()

Filepaths: 10.51 with batch_size 8  
Managed: 9.56 with batch_size 8 